### Best Location to Open a Hotel in Chennai, India

Import libraries

In [4]:
! pip install geocoder
! pip install folium==0.5.0

     |████████████████████████████████| 98 kB 7.8 MB/s  eta 0:00:01
     |████████████████████████████████| 79 kB 9.4 MB/s  eta 0:00:01
  Created wheel for folium: filename=folium-0.5.0-py3-none-any.whl size=76240 sha256=fce90e2d3b3366d045a3ed2b7dcdc8c795282b7f45a8470d5cbecc2a35195c4d
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/b2/2f/2c/109e446b990d663ea5ce9b078b5e7c1a9c45cca91f377080f8
Successfully built folium


In [5]:
import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values # to get coordinates

import requests
import geocoder# library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries  are imported.")

Libraries  are imported.


Data Scrapping

In [6]:
# send the GET request
data = requests.get("https://commons.wikimedia.org/wiki/Category:Suburbs_of_Chennai")
if data.status_code == 200:
    print('Page download successful')
else:
    print('Page download error. Error code: {}'.format(page.status_code))
data=data.text

Page download successful


In [7]:
# parse data from the html into a beautifulsoup object

soup = BeautifulSoup(data, 'html.parser')
# create a list to store neighborhood data
neighborhoodList = []
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)
    
    # create a new DataFrame from the list
ch_df = pd.DataFrame({"Neighborhood": neighborhoodList})
print("Total no of neighborhoods:",ch_df.shape)
ch_df.head()

Total no of neighborhoods: (63, 1)


,Neighborhood
0,► Adambakkam‎ (13 F)
1,"► Adyar‎ (5 C, 17 F)"
2,► Alwarthirunagar‎ (9 F)
3,"► Ambattur‎ (1 C, 10 F)"
4,"► Anna Nagar‎ (2 C, 6 F)"


Accessing Latitudes and Longitudes

In [8]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Chennai, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [9]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in ch_df["Neighborhood"].tolist() ]
coords[0:5]

[[12.99192000000005, 80.20603000000006],
 [12.97814999845724, 80.18882999205442],
 [13.050550000000044, 80.18397000000004],
 [13.129079995271326, 80.16889003113329],
 [12.976730020240282, 80.1439999696538]]

In [10]:
ch_df['Latitude'], ch_df['Longitude'] = zip(*ch_df['Neighborhood'].apply(get_latlng))
ch_df.head()

,Neighborhood,Latitude,Longitude
0,► Adambakkam‎ (13 F),12.99192,80.20603
1,"► Adyar‎ (5 C, 17 F)",12.97815,80.18883
2,► Alwarthirunagar‎ (9 F),13.05055,80.18397
3,"► Ambattur‎ (1 C, 10 F)",13.12908,80.16889
4,"► Anna Nagar‎ (2 C, 6 F)",12.97673,80.14400


In [11]:
ch_df.shape

(63, 3)

Visualization

In [12]:

# get the coordinates of Chennai
address = 'Chennai, India'

geolocator = Nominatim(user_agent="jeslyjose25@gmail.com")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Chennai, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Chennai, India 13.0836939, 80.270186.


In [13]:
# create map of Chennai using latitude and longitude values
map_ch = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(ch_df['Latitude'], ch_df['Longitude'], ch_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_ch)  
    
print("Neighborhood of Chennai")  
map_ch

Neighborhood of Chennai


Foursquare API data acquisition

In [15]:
CLIENT_ID = 'VCBQHZWKZUBJEFXEZUNDRFULG5ZZXYYGD3F5HMCUHUUBUGE0' 
CLIENT_SECRET = 'JTI3UJ5PMEEZZKKA1HV3XUHL5MWACK0VWTGU15XSSLMDDHZN'
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VCBQHZWKZUBJEFXEZUNDRFULG5ZZXYYGD3F5HMCUHUUBUGE0
CLIENT_SECRET:JTI3UJ5PMEEZZKKA1HV3XUHL5MWACK0VWTGU15XSSLMDDHZN


Retreiving the top 100 venues that are in Chennai within a radius of 500 meters

In [17]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(ch_df['Latitude'], ch_df['Longitude'], ch_df['Neighborhood']):
    
    # Pull the nearby venues
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [18]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1977, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,► Adambakkam‎ (13 F),12.99192,80.20603,Luxe Cinemas,12.991041,80.216962,Multiplex
1,► Adambakkam‎ (13 F),12.99192,80.20603,Phoenix Market City,12.991710,80.217297,Shopping Mall
2,► Adambakkam‎ (13 F),12.99192,80.20603,Rajdhani,12.991081,80.217003,Rajasthani Restaurant
3,► Adambakkam‎ (13 F),12.99192,80.20603,IMAX®,12.990639,80.216310,Multiplex
4,► Adambakkam‎ (13 F),12.99192,80.20603,Zara,12.991411,80.216854,Clothing Store


Grouping by neighborhood and get unique categories therefater

In [19]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
► Adambakkam‎ (13 F),78,78,78,78,78,78
"► Adyar‎ (5 C, 17 F)",22,22,22,22,22,22
► Alwarthirunagar‎ (9 F),31,31,31,31,31,31
"► Ambattur‎ (1 C, 10 F)",4,4,4,4,4,4
"► Anna Nagar‎ (2 C, 6 F)",5,5,5,5,5,5
"► Anna Salai‎ (4 C, 32 F)",6,6,6,6,6,6
"► Ashok Nagar, Chennai‎ (17 F)",5,5,5,5,5,5
► Assisi Nagar‎ (2 F),4,4,4,4,4,4
"► Besant Nagar‎ (3 C, 6 F)",90,90,90,90,90,90


In [21]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 169 uniques categories.


In [22]:
# print out the list of categories
venue = venues_df['VenueCategory'].unique()[:50]
venue

array(['Multiplex', 'Shopping Mall', 'Rajasthani Restaurant',
       'Clothing Store', 'Chinese Restaurant', 'Pizza Place',
       'Sandwich Place', 'Indian Restaurant', "Women's Store",
       'Snack Place', 'Hotel', 'Ice Cream Shop', 'Restaurant',
       'Vegetarian / Vegan Restaurant', 'Train Station', 'Donut Shop',
       "Men's Store", 'Café', 'Bar', 'Coffee Shop', 'Department Store',
       'Punjabi Restaurant', 'Frozen Yogurt Shop', 'Church',
       'South Indian Restaurant', 'BBQ Joint', 'Mexican Restaurant',
       'Mediterranean Restaurant', 'Italian Restaurant',
       'Fast Food Restaurant', 'Asian Restaurant', 'Burger Joint',
       'Japanese Restaurant', 'Bakery', 'Sports Bar', 'Market',
       'IT Services', 'Breakfast Spot', 'Modern European Restaurant',
       'Convenience Store', 'Train', 'Badminton Court', 'Supermarket',
       'Indie Movie Theater', 'Metro Station', 'Juice Bar', 'Lake',
       'Grocery Store', 'Gym', 'Movie Theater'], dtype=object)

One-hot Encoding

In [23]:
# one hot encoding
ch_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ch_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ch_onehot.columns[-1]] + list(ch_onehot.columns[:-1])
ch_onehot = ch_onehot[fixed_columns]

print(ch_onehot.shape)
ch_onehot.head()

(1977, 170)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Badminton Court,Bakery,Bar,Basketball Court,Beach,Bengali Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Buffet,Burger Joint,Bus Line,Bus Station,Cafeteria,Café,Campground,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Library,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cricket Ground,Department Store,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Event Space,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Historic Site,Hotel,Hotel Bar,Hyderabadi Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Lounge,Malay Restaurant,Market,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Motel,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,National Park,Nightclub,North Indian Restaurant,Office,Park,Performing Arts Venue,Persian Restaurant,Pharmacy,Pizza Place,Platform,Playground,Pool,Pool Hall,Portuguese Restaurant,Pub,Punjabi Restaurant,Rajasthani Restaurant,Recreation Center,Resort,Restaurant,River,Road,Rock Club,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shopping Mall,Snack Place,Soccer Stadium,South Indian Restaurant,Spa,Sports Bar,Stadium,Steakhouse,Supermarket,Surf Spot,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Train,Train Station,Vegetarian / Vegan Restaurant,Video Store,Whisky Bar,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,► Adambakkam‎ (13 F),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,► Adambakkam‎ (13 F),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,► Adambakkam‎ (13 F),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,► Adambakkam‎ (13 F),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,► Adambakkam‎ (13 F),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [25]:
ch_grouped = ch_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(ch_grouped.shape)
ch_grouped.head()

(63, 170)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Badminton Court,Bakery,Bar,Basketball Court,Beach,Bengali Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Buffet,Burger Joint,Bus Line,Bus Station,Cafeteria,Café,Campground,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Library,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cricket Ground,Department Store,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Event Space,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Historic Site,Hotel,Hotel Bar,Hyderabadi Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Lounge,Malay Restaurant,Market,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Motel,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,National Park,Nightclub,North Indian Restaurant,Office,Park,Performing Arts Venue,Persian Restaurant,Pharmacy,Pizza Place,Platform,Playground,Pool,Pool Hall,Portuguese Restaurant,Pub,Punjabi Restaurant,Rajasthani Restaurant,Recreation Center,Resort,Restaurant,River,Road,Rock Club,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shopping Mall,Snack Place,Soccer Stadium,South Indian Restaurant,Spa,Sports Bar,Stadium,Steakhouse,Supermarket,Surf Spot,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Train,Train Station,Vegetarian / Vegan Restaurant,Video Store,Whisky Bar,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,► Adambakkam‎ (13 F),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.012821,0.0,0.0,0.025641,0.012821,0.012821,0.012821,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.012821,0.0,0.012821,0.0,0.0,0.0,0.038462,0.0,0.025641,0.0,0.012821,0.051282,0.0,0.038462,0.0,0.0,0.0,0.0,0.0,0.012821,0.0,0.0,0.038462,0.0,0.0,0.00,0.012821,0.0,0.0,0.0,0.0,0.025641,0.0,0.0,0.0,0.0,0.0,0.0,0.012821,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.051282,0.0,0.0,0.012821,0.012821,0.115385,0.000000,0.012821,0.0,0.012821,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.012821,0.012821,0.0,0.012821,0.000000,0.012821,0.0,0.012821,0.0,0.0,0.000000,0.0,0.025641,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.076923,0.0,0.0,0.0,0.0,0.0,0.0,0.012821,0.012821,0.0,0.0,0.025641,0.00,0.0,0.0,0.0,0.038462,0.0,0.0,0.0,0.012821,0.012821,0.0,0.012821,0.0,0.012821,0.0,0.0,0.012821,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.012821,0.012821,0.051282,0.0,0.0,0.012821,0.0,0.0,0.0
1,"► Adyar‎ (5 C, 17 F)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.045455,0.0,0.0,0.000000,0.045455,0.045455,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.045455,0.0,0.000000,0.0,0.0,0.0,0.090909,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.045455,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.045455,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.045455,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.045455,0.000000,0.0,0.0,0.0,0.0,0.136364,0.0,0.0,0.000000,0.000000,0.090909,0.045455,0.000000,0.0,0.000000,0.0,0.045455,0.0,0.0,0.0,0.045455,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.045455,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0

In [26]:
len(ch_grouped[ch_grouped["Hotel"] > 0])

25

In [29]:
ch_hotel = ch_grouped[["Neighborhoods","Hotel"]]
ch_hotel.head()

,Neighborhoods,Hotel
0,► Adambakkam‎ (13 F),0.051282
1,"► Adyar‎ (5 C, 17 F)",0.136364
2,► Alwarthirunagar‎ (9 F),0.000000
3,"► Ambattur‎ (1 C, 10 F)",0.000000
4,"► Anna Nagar‎ (2 C, 6 F)",0.000000


Clustering neighborhoods

In [30]:
kclusters = 4

ch_clustering = ch_hotel.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ch_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 3, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [31]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
ch_merged = ch_hotel.copy()

# add clustering labels
ch_merged["Cluster Labels"] = kmeans.labels_

In [32]:
ch_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
ch_merged.head()

,Neighborhood,Hotel,Cluster Labels
0,► Adambakkam‎ (13 F),0.051282,0
1,"► Adyar‎ (5 C, 17 F)",0.136364,3
2,► Alwarthirunagar‎ (9 F),0.000000,2
3,"► Ambattur‎ (1 C, 10 F)",0.000000,2
4,"► Anna Nagar‎ (2 C, 6 F)",0.000000,2


In [33]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
ch_merged = ch_merged.join(ch_df.set_index("Neighborhood"), on="Neighborhood")

print(ch_merged.shape)
ch_merged.head() # check the last columns!

(63, 5)


,Neighborhood,Hotel,Cluster Labels,Latitude,Longitude
0,► Adambakkam‎ (13 F),0.051282,0,12.99192,80.20603
1,"► Adyar‎ (5 C, 17 F)",0.136364,3,12.97815,80.18883
2,► Alwarthirunagar‎ (9 F),0.000000,2,13.05055,80.18397
3,"► Ambattur‎ (1 C, 10 F)",0.000000,2,13.12908,80.16889
4,"► Anna Nagar‎ (2 C, 6 F)",0.000000,2,12.97673,80.14400


In [34]:
print(ch_merged.shape)
ch_merged.sort_values(["Cluster Labels"], inplace=True)
ch_merged.head()

(63, 5)


,Neighborhood,Hotel,Cluster Labels,Latitude,Longitude
0,► Adambakkam‎ (13 F),0.051282,0,12.99192,80.20603
57,► Valmiki Nagar‎ (4 F),0.045455,0,12.98139,80.26377
54,"► Triplicane‎ (3 C, 26 F)",0.050000,0,13.06289,80.27146
52,"► Thiruvanmiyur‎ (1 C, 12 F)",0.034483,0,12.98619,80.26051
44,"► Semmencherry‎ (1 C, 1 F)",0.047619,0,12.86557,80.22051


Visualization

In [35]:
import numpy as np
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ch_merged['Latitude'], ch_merged['Longitude'], ch_merged['Neighborhood'], ch_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [36]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

Filtering clusters

First Cliuster (0)

In [37]:
ch_merged.loc[ch_merged['Cluster Labels'] == 0]

,Neighborhood,Hotel,Cluster Labels,Latitude,Longitude
0,► Adambakkam‎ (13 F),0.051282,0,12.99192,80.20603
57,► Valmiki Nagar‎ (4 F),0.045455,0,12.98139,80.26377
54,"► Triplicane‎ (3 C, 26 F)",0.050000,0,13.06289,80.27146
52,"► Thiruvanmiyur‎ (1 C, 12 F)",0.034483,0,12.98619,80.26051
44,"► Semmencherry‎ (1 C, 1 F)",0.047619,0,12.86557,80.22051
21,"► Kotturpuram‎ (2 C, 4 F)",0.051546,0,13.01696,80.24276
20,► Kosapet‎ (8 F),0.063830,0,13.09453,80.25482
13,"► Egmore‎ (5 C, 5 F)",0.050000,0,13.07642,80.25756
31,"► Nungambakkam‎ (3 C, 7 F)",0.040000,0,13.06160,80.24315


Second Cluster (1)

In [38]:
ch_merged.loc[ch_merged['Cluster Labels'] == 1]

,Neighborhood,Hotel,Cluster Labels,Latitude,Longitude
60,► Villivakkam‎ (1 C),0.250000,1,13.13433,80.20618
29,► Nanganallur‎ (3 F),0.187500,1,12.97640,80.18760
27,"► Muttukadu, Chennai‎ (3 C, 1 F)",0.187500,1,12.83165,80.24207
47,"► St. Thomas Mount‎ (2 C, 41 F)",0.166667,1,13.00799,80.19599


Third Cluster (2)

In [39]:
ch_merged.loc[ch_merged['Cluster Labels'] == 2]

,Neighborhood,Hotel,Cluster Labels,Latitude,Longitude
9,"► Chepauk‎ (3 C, 6 F)",0.014706,2,13.064400,80.280650
36,"► Perambur‎ (3 C, 15 F)",0.000000,2,13.122470,80.235690
37,► Perungudi‎ (8 F),0.000000,2,12.963560,80.240010
38,"► Poonamallee‎ (2 C, 4 F)",0.000000,2,13.051370,80.112670
39,► Puzhal‎ (2 C),0.000000,2,13.159460,80.207180
40,"► Ramapuram‎ (1 C, 8 F)",0.000000,2,13.109980,80.152860
42,"► Royapuram‎ (1 C, 6 F)",0.000000,2,13.113940,80.294200
4,"► Anna Nagar‎ (2 C, 6 F)",0.000000,2,12.976730,80.144000
46,► Sithalapakkam‎ (7 F),0.000000,2,12.890090,80.184590
8,"► Besant Nagar‎ (3 C, 6 F)",0.011111,2,13.000170,80.257660


Fourth Cluster (3)

In [40]:
ch_merged.loc[ch_merged['Cluster Labels'] == 3]

,Neighborhood,Hotel,Cluster Labels,Latitude,Longitude
12,► Covelong‎ (23 F),0.076923,3,12.78984,80.24914
1,"► Adyar‎ (5 C, 17 F)",0.136364,3,12.97815,80.18883
56,"► Vadapalani‎ (2 C, 10 F)",0.072464,3,13.05226,80.21120
28,"► Mylapore‎ (3 C, 16 F)",0.110000,3,13.03155,80.26022
25,"► Meenambakkam‎ (1 C, 1 F)",0.078947,3,12.98646,80.17600
16,"► Guindy‎ (4 C, 1 P, 17 F)",0.086957,3,13.00408,80.22012
41,"► Royapettah‎ (2 C, 1 F)",0.080000,3,13.05352,80.26826
43,► Saidapet‎ (20 F),0.081967,3,13.02027,80.22131
55,► Urapakkam‎ (3 F),0.071429,3,12.86342,80.06916
48,"► T. Nagar‎ (1 C, 1 P, 57 F)",0.074468,3,13.04536,80.23390


Conclusion

Conclusively, after k-means clustering, it can be inferred that the second Cluster is the best suited location as the least number of hotels are in these areas. Apparently, cluster 1 areas have good proximity to transport networks and popular places for tourist attraction. 